In [1]:
pid_index_range= [0, 1]
number_of_atlas = 5;
number_of_train_files = number_of_atlas - 1# having 5 for testing

In [2]:
import read_atlas_layer_points as rlp

main_folder = '/iacl/pg17/ja/data/20171205_cirrus_manual_results/'

#filename = 'C:/D/dev/Xnorm/experiments/oct/20171204_manual_segmentation/filenames.txt'
filename = '/home/ja/dev/Xnorm/experiments/oct/20171204_manual_segmentation/filenames.txt'
pids, image_prefixes = rlp.read_manual_segmentation_oct_files(filename)

#filename = 'C:/D/dev/Xnorm/experiments/oct/20170830_large_healthy_data_proc/cirrus_od_altas_folder_linux_iacl.txt'
filename = '/home/ja/dev/Xnorm/experiments/oct/20170830_large_healthy_data_proc/cirrus_od_altas_folder_linux_iacl.txt'
atlas_folders, atlas_subfolders = rlp.read_atlas_images_files(filename)

print('PID: {}\nImagePrefixes:{}'.format(pids, image_prefixes))
print('Atlas folders:{}\nAtlas subfolders:{}'.format(atlas_folders, atlas_subfolders))

PID: ['P1067763439', 'P1092977057', 'P1103182673', 'P1103946697', 'P1132308285', 'P1210931577']
ImagePrefixes:['P1067763439_Macular Cube 512x128_9-15-2011_15-48-40_OD_sn5701_cube_z', 'P1092977057_Macular Cube 512x128_2-15-2012_16-13-29_OS_sn10264_cube_z', 'P1103182673_Macular Cube 512x128_9-15-2011_10-10-55_OD_sn5737_cube_z', 'P1103946697_Macular Cube 512x128_10-20-2011_13-27-42_OD_sn4534_cube_z', 'P1132308285_Macular Cube 512x128_4-21-2010_8-58-53_OD_sn10122_cube_z', 'P1210931577_Macular Cube 512x128_2-20-2012_12-2-34_OD_sn11344_cube_z']
Atlas folders:['/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0003', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0005', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0006_', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0009', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0010', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0011', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/CC_0012', '/iacl/pg17/ja/data/20170712_oct_ms_calabresi/C

In [3]:
import os

deformed_layer_pnt_filename = 'deformed_layer_points.csv';
# each patient having dict of label as a key & a list of points as a value
deformed_layer_points_per_pid = dict()

for number, pid in enumerate(pids):
    if number not in pid_index_range: continue
    print('Number ', number, ' pid ', pid)
    filenames = []
    for indx, atlas_subfolder in enumerate(atlas_subfolders):
        if indx >= number_of_atlas: 
            print('Using %d atlases' % indx)
            break;
        filename = '/'.join([pid, atlas_subfolder, deformed_layer_pnt_filename])
        filename = os.path.join(main_folder, filename)
        filenames.append(filename)
    print('Using %d atlases' % len(filenames))
    deformed_layer_points_per_pid[pid] = rlp.get_atlas_layer_points(filenames)
    
deformed_layer_points_per_pid[pids[pid_index_range[0]]]

Number  0  pid  P1067763439
Using 5 atlases
Using 5 atlases
Number  1  pid  P1092977057
Using 5 atlases
Using 5 atlases


{'1': array([[ 8.05157e-01, -1.60219e-03, -3.55315e-01],
        [ 8.04476e-01,  1.00966e-02, -3.55312e-01],
        [ 8.03831e-01,  2.17955e-02, -3.55309e-01],
        ...,
        [ 7.11695e-01,  5.97935e+00,  5.89792e+00],
        [ 6.85760e-01,  5.99105e+00,  5.89792e+00],
        [ 6.85903e-01,  6.00275e+00,  5.89792e+00]], dtype=float32),
 '2': array([[ 8.21524e-01, -1.57339e-03, -3.55299e-01],
        [ 8.20814e-01,  1.01254e-02, -3.55296e-01],
        [ 8.20201e-01,  2.18243e-02, -3.55293e-01],
        ...,
        [ 8.05479e-01,  5.97934e+00,  5.89805e+00],
        [ 7.92858e-01,  5.99104e+00,  5.89806e+00],
        [ 7.92951e-01,  6.00274e+00,  5.89806e+00]], dtype=float32),
 '3': array([[ 8.66533e-01, -1.49419e-03, -3.55257e-01],
        [ 8.66043e-01,  1.02050e-02, -3.55254e-01],
        [ 8.65389e-01,  2.19038e-02, -3.55251e-01],
        ...,
        [ 8.47341e-01,  5.97934e+00,  5.89812e+00],
        [ 8.44323e-01,  5.99104e+00,  5.89812e+00],
        [ 8.44443e-01,  6.00

In [6]:
for pid, atlas_layer_points in deformed_layer_points_per_pid.items(): 
    print('Data shape: ', atlas_layer_points['1'].shape, ' number of points: ', 512*128*len(atlas_subfolders))

Data shape:  (327680, 3)  number of points:  2031616
Data shape:  (327680, 3)  number of points:  2031616


In [9]:
import time
import numpy as np

from sklearn.externals import joblib
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import Matern

number_of_points_per_file = 512*128

# fit the kernel for each reference image, i.e., ref pid
for pid, atlas_layer_points in deformed_layer_points_per_pid.items(): 
    
    logfilename = os.path.join(main_folder, pid, 'kernel_estimate_log.txt')
    
    # fit the kernel to each layer boundary
    for label, layer_points in atlas_layer_points.items():
        print('\n-----------------PID: {}, Label:{}----------------------'.format(pid, label))
        X_train = layer_points[:number_of_train_files*number_of_points_per_file, 1:]
        y_train = layer_points[:number_of_train_files*number_of_points_per_file, 0]
        
        X_test = layer_points[number_of_train_files*number_of_points_per_file:, 1:]
        y_test = layer_points[number_of_train_files*number_of_points_per_file:, 0]
        
        # set up hyperparameter selection 
        param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
                      "kernel": [Matern(length_scale=l, nu=2.5) for l in np.logspace(-3, 0, 20)]}
        kr = GridSearchCV(KernelRidge(), cv=5, param_grid=param_grid, n_jobs=1, 
                  verbose=100, return_train_score=False)
        
        # fit kernel to data 
        t0 = time.time()
        kr.fit(X_train, y_train)
        fit_time = time.time() - t0
        print('KRR complexity and bandwidth selected and model fitted in %.3f s' % (time.time() - t0))
        
        #save the fitted kernel 
        print('Save kernel %s' % pid)
        filename = 'matern_kernel_label{}.pkl'.format(label)
        filename = os.path.join(main_folder, pid, filename)
        joblib.dump(kr, filename)  # to load, kr = joblib.load(filename) 
        
        # accuracy on test data
        accuracy_on_test_data = kr.score(X_test, y_test)
        
        log_info = ('PID: %s, Label: %s\n' % (pid, label)) +\
                   ('\tBest training score: %.4f\n' %  kr.best_score_)+\
                   ('\tBest test score: %.4f\n' % accuracy_on_test_data)+\
                   ('\tBest kernel: %s\n' %  kr.best_estimator_.kernel)+\
                    ('\tBest length scale: %.4f\n' %  kr.best_estimator_.length_scale)+\
                   '\tBest length scale bound: {}\n'.format(kr.best_estimator_.length_scale_bounds)
                
        with open(logfilename, mode='a', encoding='utf-8') as afile:
            afile.write(log_info)
        
        


-----------------PID: P1092977057, Label:9----------------------
Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV] kernel=Matern(length_scale=0.001, nu=2.5), alpha=1.0 ............


MemoryError: 